In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')
filepath="/content/drive/MyDrive/Colab Notebooks"

Mounted at /content/drive


In [15]:
stock_data = pd.read_csv(f"{filepath}/VNIndex_data.csv")
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1252 entries, 0 to 1251
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Attributes    1252 non-null   object
 1   code          1251 non-null   object
 2   high          1251 non-null   object
 3   low           1251 non-null   object
 4   open          1251 non-null   object
 5   close         1251 non-null   object
 6   adjust        1251 non-null   object
 7   volume_match  1251 non-null   object
 8   value_match   1251 non-null   object
dtypes: object(9)
memory usage: 88.2+ KB


In [16]:
stock_data.head()

,Attributes,code,high,low,open,close,adjust,volume_match,value_match
0,Symbols,VNINDEX,VNINDEX,VNINDEX,VNINDEX,VNINDEX,VNINDEX,VNINDEX,VNINDEX
1,date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-12-31,VNINDEX,1272.46,1266.78,1272.25,1266.78,1266.78,407171783.0,9570643132850.0
3,2024-12-30,VNINDEX,1274.15,1268.77,1273.75,1272.02,1272.02,452946957.0,10012327206260.0
4,2024-12-27,VNINDEX,1277.89,1272.23,1275.56,1275.14,1275.14,579882989.0,13286059082390.0


# **Step 1: Data Cleaning + Transformation**


In [17]:
stock_data = stock_data.drop([0, 1])  # Xóa dòng 0, 1, 2 và dòng cuối cùng
stock_data.rename(columns={'Attributes': 'date'}, inplace=True)
stock_data.reset_index(drop=True, inplace=True)


In [18]:
# Kiểm tra missing values
missing_values = stock_data.isnull().sum()
print("Missing Values:\n", missing_values)

# Kiểm tra duplicate rows
duplicate_rows = stock_data.duplicated().sum()
if duplicate_rows > 0:
    # Remove duplicate rows
    stock_data.drop_duplicates(inplace=False)
    print(f'Removed {duplicate_rows} duplicate rows')
else:
    print('No duplicate rows found')


# Check for outliers in numerical columns
numerical_summary = stock_data.describe()
print("\nNumerical Summary:\n", numerical_summary)

Missing Values:
 date            0
code            0
high            0
low             0
open            0
close           0
adjust          0
volume_match    0
value_match     0
dtype: int64
No duplicate rows found

Numerical Summary:
               date     code     high      low     open    close   adjust  \
count         1250     1250     1250     1250     1250     1250     1250   
unique        1250        1     1230     1240     1228     1241     1241   
top     2019-12-31  VNINDEX  1058.14  1222.92  1338.11  1279.91  1279.91   
freq             1     1250        2        2        2        2        2   

       volume_match      value_match  
count          1250             1250  
unique         1250             1250  
top     141842010.0  2194970000000.0  
freq              1                1  


In [19]:
stock_data

,date,code,high,low,open,close,adjust,volume_match,value_match
0,2024-12-31,VNINDEX,1272.46,1266.78,1272.25,1266.78,1266.78,407171783.0,9570643132850.0
1,2024-12-30,VNINDEX,1274.15,1268.77,1273.75,1272.02,1272.02,452946957.0,10012327206260.0
2,2024-12-27,VNINDEX,1277.89,1272.23,1275.56,1275.14,1275.14,579882989.0,13286059082390.0
3,2024-12-26,VNINDEX,1277.17,1271.99,1275.68,1272.87,1272.87,502071890.0,10640383260640.0
4,2024-12-25,VNINDEX,1283.09,1261.87,1262.03,1274.04,1274.04,724345690.0,16511995715950.0
...,...,...,...,...,...,...,...,...,...
1245,2020-01-07,VNINDEX,959.46,953.19,955.79,958.88,958.88,126566780.0,2072200000000.0
1246,2020-01-06,VNINDEX,965.14,955.59,962.79,955.79,955.79,137059570.0,2227952000000.0
1247,2020-01-03,VNINDEX,970.88,965.14,968.72,965.14,965.14,144668880.0,2447689000000.0
1248,2020-01-02,VNINDEX,966.67,959.67,960.26,966.67,966.67,128410670.0,2230019850000.0


Kết quả kiểm tra trong bước Data Cleaning:
- Không có giá trị thiếu trong bất kỳ cột nào trừ các ngày làm việc
- Không có dòng trùng lặp trong dữ liệu.
- Các giá trị số có vẻ hợp lý, không xuất hiện bất thường rõ ràng.

In [20]:
# Convert 'date' column to pandas datetime format
stock_data['date'] = pd.to_datetime(stock_data['date'])  # Convert to datetime
stock_data.set_index('date', inplace=True)  # Set as index, so that pandas can handle timeseries
stock_data = stock_data.sort_index()  # Ensure it's sorted

In [21]:
# Fill missing data only for weekdays (Monday to Friday)
stock_data_filled = stock_data.asfreq('B').copy()  # asfreq('B') auto keep business day and fill missing data, remove duplicate day

# Fill missing values (which includes holidays) with the previous valid value
stock_data_filled = stock_data_filled.fillna(method='ffill')

print("\nFilled dataframe (only business days, with holidays filled):")
print(stock_data_filled.head())

# Display the full filled dataframe
stock_data_filled


Filled dataframe (only business days, with holidays filled):
               code    high     low    open   close  adjust volume_match  \
date                                                                       
2019-12-31  VNINDEX  965.71  958.22  965.71  960.99  960.99  141842010.0   
2020-01-01  VNINDEX  965.71  958.22  965.71  960.99  960.99  141842010.0   
2020-01-02  VNINDEX  966.67  959.67  960.26  966.67  966.67  128410670.0   
2020-01-03  VNINDEX  970.88  965.14  968.72  965.14  965.14  144668880.0   
2020-01-06  VNINDEX  965.14  955.59  962.79  955.79  955.79  137059570.0   

                value_match  
date                         
2019-12-31  2194970000000.0  
2020-01-01  2194970000000.0  
2020-01-02  2230019850000.0  
2020-01-03  2447689000000.0  
2020-01-06  2227952000000.0  


<ipython-input-21-be88481e95ee>:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stock_data_filled = stock_data_filled.fillna(method='ffill')


,code,high,low,open,close,adjust,volume_match,value_match
date,,,,,,,,
2019-12-31,VNINDEX,965.71,958.22,965.71,960.99,960.99,141842010.0,2194970000000.0
2020-01-01,VNINDEX,965.71,958.22,965.71,960.99,960.99,141842010.0,2194970000000.0
2020-01-02,VNINDEX,966.67,959.67,960.26,966.67,966.67,128410670.0,2230019850000.0
2020-01-03,VNINDEX,970.88,965.14,968.72,965.14,965.14,144668880.0,2447689000000.0
2020-01-06,VNINDEX,965.14,955.59,962.79,955.79,955.79,137059570.0,2227952000000.0
...,...,...,...,...,...,...,...,...
2024-12-25,VNINDEX,1283.09,1261.87,1262.03,1274.04,1274.04,724345690.0,16511995715950.0
2024-12-26,VNINDEX,1277.17,1271.99,1275.68,1272.87,1272.87,502071890.0,10640383260640.0
2024-12-27,VNINDEX,1277.89,1272.23,1275.56,1275.14,1275.14,579882989.0,13286059082390.0


In [22]:
# Lưu lại giá trị của ngày 01-01-2020 trước khi xóa
backup_row = stock_data_filled.loc['2020-01-01'].copy()

# Xóa ngày 31-12-2019
stock_data_filled = stock_data_filled.drop(index='2019-12-31')

# Khôi phục ngày 01-01-2020 nếu nó bị mất do ffill trước đó
if '2020-01-01' not in stock_data_filled.index:
    stock_data_filled.loc['2020-01-01'] = backup_row

# Sort lại index để đảm bảo thứ tự đúng
stock_data_filled = stock_data_filled.sort_index()
stock_data_filled

,code,high,low,open,close,adjust,volume_match,value_match
date,,,,,,,,
2020-01-01,VNINDEX,965.71,958.22,965.71,960.99,960.99,141842010.0,2194970000000.0
2020-01-02,VNINDEX,966.67,959.67,960.26,966.67,966.67,128410670.0,2230019850000.0
2020-01-03,VNINDEX,970.88,965.14,968.72,965.14,965.14,144668880.0,2447689000000.0
2020-01-06,VNINDEX,965.14,955.59,962.79,955.79,955.79,137059570.0,2227952000000.0
2020-01-07,VNINDEX,959.46,953.19,955.79,958.88,958.88,126566780.0,2072200000000.0
...,...,...,...,...,...,...,...,...
2024-12-25,VNINDEX,1283.09,1261.87,1262.03,1274.04,1274.04,724345690.0,16511995715950.0
2024-12-26,VNINDEX,1277.17,1271.99,1275.68,1272.87,1272.87,502071890.0,10640383260640.0
2024-12-27,VNINDEX,1277.89,1272.23,1275.56,1275.14,1275.14,579882989.0,13286059082390.0


# **Dimensionality Reduction**



In [24]:
# Bỏ các cột không liên quan trực tiếp đến dự đoán giá cổ phiếu
columns_to_drop = ['code', 'adjust', 'value_match']
stock_data_reduced = stock_data_filled.drop(columns=columns_to_drop, axis=1)

stock_data_reduced.head()

,high,low,open,close,volume_match
date,,,,,
2020-01-01,965.71,958.22,965.71,960.99,141842010.0
2020-01-02,966.67,959.67,960.26,966.67,128410670.0
2020-01-03,970.88,965.14,968.72,965.14,144668880.0
2020-01-06,965.14,955.59,962.79,955.79,137059570.0
2020-01-07,959.46,953.19,955.79,958.88,126566780.0


In [26]:
from google.colab import files
# Save the filled dataframe to a CSV file

stock_data_reduced.to_csv('Stock_VNIndex_filled.csv')

# Download the filled CSV file
files.download('Stock_VNIndex_filled.csv') # Use files.download instead of stock_data_reduced.download

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>